<a href="https://colab.research.google.com/github/MhmudAlpurd/AnimationApp/blob/master/hwrs002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/My Drive/

/content/drive/My Drive


In [4]:
import os, cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import random as rnd
from tqdm import tqdm
import pickle as pkl
import PIL

**#path creation**


In [5]:
DIRECTORY = 'Hampalai/HWRS_0_0_2/'
CATEGORIES = ['plastics', 'metal' , 'paper' , 'glass']
img_lbl_arr = []
IMAGE_SIZE = 224

In [6]:
for category in tqdm(CATEGORIES):
    category_folder = os.path.join(DIRECTORY, category)
    img_label = CATEGORIES.index(category)
    for img in os.listdir(category_folder):
        image_path = os.path.join(category_folder, img)
        #img_read = mpimg.imread(image_path)
        #imgplot = plt.imshow(img_read)
        #plt.show()
        img_loaded_r = cv2.imread(image_path)
        img_loaded = cv2.resize(img_loaded_r, (IMAGE_SIZE,IMAGE_SIZE))
        img_lbl_arr.append([img_loaded, img_label])

print(len(img_lbl_arr))

100%|██████████| 4/4 [00:11<00:00,  2.92s/it]

1987


**#shuffle the data**

In [7]:
rnd.shuffle(img_lbl_arr)

**#split image and labels and convert lst to arr**

In [8]:
X = []
y = []

for img_tensor, labels in img_lbl_arr:
    X.append(img_tensor) #X = image tensor
    y.append(labels) #y = label list

X = np.array(X)
y = np.array(y)

#pkl.dump(X, open('X.pkl', 'wb'))
#pkl.dump(y, open('y.pkl', 'wb'))

#X = pkl.load(open('X.pkl', 'rb'))
#y = pkl.load(open('y.pkl', 'rb'))

print(X.shape)
#(1987, 150, 150, 3)


(1987, 224, 224, 3)


In [9]:
from sklearn.model_selection import  train_test_split
from tensorflow.keras.utils import to_categorical

img_train, img_test, lbl_train, lbl_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
print('img_train:', len(img_train), type(img_train[0]), '| img_test:', len(img_test), '| lbl_train:', len(lbl_train), type(lbl_train[0]) , '| lbl_test:', len(lbl_test))

# convert the labels from integers to vectors
lbl_train = to_categorical(lbl_train, num_classes=4)
lbl_test = to_categorical(lbl_test, num_classes=4)


img_train: 1589 <class 'numpy.ndarray'> | img_test: 398 | lbl_train: 1589 <class 'numpy.int64'> | lbl_test: 398


**#ImageDataGenerator**

In [21]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True, samplewise_center=True,
    featurewise_std_normalization=True, samplewise_std_normalization=True,
    rotation_range=20, width_shift_range=0.2,
    height_shift_range=0.2, shear_range=0.1, zoom_range=0.2,
    channel_shift_range=0.2, fill_mode='nearest', cval=0.1, horizontal_flip=True,
    vertical_flip=True , rescale=True

)

datagen.fit(img_train)


**#model creation**

In [ ]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.optimizers import SGD

#base_model = VGG16(include_top= True, weights='imagenet', input_shape=(IMAGE_SIZE,IMAGE_SIZE,3), pooling= 'avg', classes= 1000 , classifier_activation = None)
#input_layer = Input(base_model)

num_classes = 4

myModel = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])


from keras.optimizers import SGD, Adam
from keras.losses import categorical_crossentropy
optimizer_1 = SGD(lr=1e-4, momentum=0.99)
optimizer_2 = Adam(lr=1e-3)

#https://www.pyimagesearch.com/2017/12/11/image-classification-with-keras-and-deep-learning/

#myModel.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
myModel.compile(loss="binary_crossentropy", optimizer='Adam', metrics=["accuracy"])
myModel.summary()

myModel.fit(x=datagen.flow(img_train, lbl_train, batch_size=32),validation_data=(img_test, lbl_test), steps_per_epoch=len(img_train) // 32,epochs=96, verbose=1)


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_5 (Rescaling)      (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 224, 224, 16)      448       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 112, 112, 32)      4640      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 56, 56, 64)        18496     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 28, 28, 64)       

In [ ]:
from keras.optimizers import SGD, Adam
from keras.losses import categorical_crossentropy
optimizer_1 = SGD(lr=1e-4, momentum=0.99)
optimizer_2 = Adam(lr=1e-3)
model.compile(loss='binary_crossentropy', optimizer=optimizer_1, metrics=['accuracy'])


EPOCHS = 100
BS = 32
# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")
# train the network
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS)
  

In [ ]:
#https://www.pyimagesearch.com/2018/12/24/how-to-use-keras-fit-and-fit_generator-a-hands-on-tutorial/
#The Keras deep learning libraries to train model:
#1.fit
#2.fit_generator
#3.train_on_batch

EPOCHS = 100
BS = 128

trained_model = myModel.fit(X, y ,batch_size= BS, epochs= EPOCHS,verbose=2, validation_split= 0.2, validation_data =(img_test, lbl_test) , validation_steps = 100,steps_per_epoch=100, shuffle=True)

#history = myModel.fit(
   # image_gen.flow(img_train, lbl_train, batch_size=BS, shuffle= True),
   # validation_data =(img_test, lbl_test),
    #validation_steps = 100,
    #steps_per_epoch =  (len(img_train) // BS ),
   # epochs = EPOCHS,
    #verbose= 1)


**#Fitting**

In [ ]:
input_layer = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same')(input_layer)
pool1 = MaxPooling2D(pool_size = 2)(conv1)
conv2 = Conv2D(32, 3, activation = 'relu', padding = 'same')(pool1)
pool2 = MaxPooling2D(pool_size=2)(conv2)
flat = Flatten()(pool2)
out_layer = Dense(4, activation = 'softmax')(flat)
myModel = Model(input_layer, out_layer)
myModel.summary()

**#Prediction**